In [2]:
## Install all the required packages and run the required libraries
##-----------------------------------------------
import pandas as pd
import numpy as np

In [3]:
## List of Functions
##-----------------------------------------------
#read a CSV file
#readCSV(String file):The readCSV function reads the CSV file into workplace
#Arguments: file is the CSV file in the select work directory.
def readCSV(filename):
    csv_data = pd.read_csv(filename)
    return csv_data

In [19]:
#write to CSV file
#writeCSV(table DataSet, String file):The writeCSV function writes the dataframe to a CSV file.
#Arguments: DataSet is the name of dataframe to save, file is the name of the CSV output file.
#Returns a CSV file
def writeCSV(dataset,filename):
     dataset.to_csv(filename)

In [12]:
#select dataset columns for analysis
#selectColumns(table DataSet, string columnName, …): This function selects/keeps the list columns needed for analysis from the dataset. Only the
#list of selected columns/attributes are included in the dataset. 
#Arguments: DataSet is the name of the dataframe, columnName is the name of the column to keep in the dataset. Many can be listed, separated by commas.
#Returns a dataset including only the list of columns/attributes that are selected
def selectColumns(dataset,selectCol):
    return dataset[selectCol]


In [13]:
#delete columns from the dataset
def deleteColumns(dataset,deleteCol):
    return dataset.drop(columns=deleteCol)